# Variable and Function initiation

## Define variables J, z, s,t
J: spin $\in 2\mathbb{Z}^+$ <br>
z$=1/m^2$ : inverse energy $\in (0,1)$ <br>
s,t: Mandelstam variables $\in (-1,0)$ subject to the constraint $s+t>-1$ <br>
For $K^{\beta}$, we have $t \in (-1,0)$

In [1]:
import numpy as np
import math
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from random import seed
from random import random
import array as arr

# J,z
z = np.arange(0.01,0.99,1/100)
N = len(z)
J = np.arange(0,100,2.0)
print("Shape of z is {}, while shape of J is {}".format(z.shape,J.shape))

# generate random floating point values for s and t
# Used for M(s,t)
# Set number of s,t
length_st=500
seed(1)
s = arr.array('f')
t = arr.array('f')

for _ in range(length_st):
    # generate random numbers between (-1,0) for s
    # Restrict between -(0.1,0.9) to avoid possible boundary singularities
    value_s=-np.random.uniform(low=0.1,high=0.9)
    s.append(value_s)
    
    # make t given restriction s+t> -1
    # further restrict between -(0.1,0.9) to avoid possible boundary singularities
    value_t = np.random.uniform(low=-value_s-0.9, high=0.1)
    t.append(value_t)
    
s = np.array(s)
t = np.array(t)

#generate random momentum transfers t E (-1,0)
# Used for K^\beta
t_individual= -1*np.random.rand(length_st)
print("Shape of s and t is {}, while shape of t_individual is {}".format(s.shape,t_individual.shape))

Shape of z is (98,), while shape of J is (50,)
Shape of s and t is (500,), while shape of t_individual is (500,)


## Functions used in Loss Function
The Loss function is defined as
$$ L[F] = |K \cdot F|^2 + \alpha |K^{\alpha} \cdot F - c |^2 - \beta K^{\beta} \cdot F,$$
where the index notation discretizes the Mandelstam variables $s_i,t_j$. We refer to the first term as the "Crossing symmetry constraint", the second as the "Einstein Gravity minimisation constraint" and the last one as the "Einstein Gravity maximisation".
<br>
<br>
The individual consistuents of this equation include:
$$K(s,t;J,z) = \frac{2}{\pi} \frac{(1+z t)(2+z t)}{(1-z t)(1+z(s+t))s(s+t)} \tilde{P}_J(1+2zt), $$
$$K^{\alpha}(t;J,z) = -\frac{2t}{\pi} z^{\frac{d}{2}-1} (2+t z) \tilde{P}_J(1+2zt),$$
$$K^{\beta}(J,z) = \frac{4}{\pi} z^{\frac{d}{2}+1} \tilde{P}_J(1).$$

The function $\tilde{P}_J(x)$ is defined as
$$\tilde{P}_J(x) = 16\pi(2J+1)P_J(x), \qquad d=4,$$
where $P_J(x)$ is a Legendre Polynomial.
<br> <br>
Below, we define $M$ as the 2D array $K_{ij}$ labelling $K(s_i,t_i;J,z)$.

In [2]:
# Functions defining K's
import scipy
import scipy.integrate as integrate
from scipy.integrate import quad
import scipy.special

#Calculate M given array of z and J, for specific points (s,t)
def Mst_Mts(s,t,z,J):
    return (M_withoutF(s,t,z,J) - M_withoutF(t,s,z,J))/N

#K_{ij} // M(s,t) matrix
def M_withoutF(s,t,z,J):   # use eq 2.6
    sum = 0
    A= []
    B = []
    for j in range(len(J)):
        for Z in z:
            a = 2/math.pi
            b = a*Z
            c = b*(1+Z*t)*(2+Z*t)
            d = c*np.array(P_J(1+2*Z*t, j))
            e = (1-Z*t)*(1+Z*(s+t))*s*(s+t)
            sum += d/e
            B.append(sum)

    B= np.array(B)
    return B

def P_J(x, J):
    B = scipy.special.lpmv(0, J, x)
    C = 16*math.pi*(2*J +1)
    D = C*B
    return D

def Gamma(z):
    return scipy.special.gamma(z)

def hypergeometric(a,b,c,d):
    return scipy.special.hyp2f1(a,b,c,d)

#K^\alpha vector
def K_alpha(t, J, z):
    sum = 0
    k = []
    for j2 in J[0:]:
        for z2 in z[0:]:
            
            A = (-2*t/math.pi)
            B = A*z2
            C = B*(2 +t*z2)
            D = C*(P_J(1+2*z2*t, j2))
            sum = D
            k.append(sum)
            sum = 0
    
    return k

#K^{\beta} vector
def K_Beta(J,z):
    sum =0
    A = []
    for j in J:
        for Z in z:
            sum += (4/math.pi)*Z**(3)*P_J(1,j)
            A.append(sum/N)
        
    return A

In [3]:
#K initialization block // Set as global variables
#Kij
M=[]
for i in range(len(s)):
    M.append(Mst_Mts(s[i],t[i],z,J))   
    
M= np.array(M)  
M =tf.convert_to_tensor(M, tf.float64)

#KA
K_A = []
for t1 in t_individual:
    value = K_alpha(t1,J,z)
    #should be appending vectors
    K_A.append(value)
K_A= np.array(K_A)
K_A =tf.convert_to_tensor(K_A, tf.float64)

#KB
K_B = K_Beta(J,z)
K_B= np.array(K_B)
K_B = np.transpose(K_B)
K_B =tf.convert_to_tensor(K_B, tf.float64)

In [6]:
#Loss function block
def crossing_Symmetry_Square(M, y_pred):
    Mf = tf.tensordot(M, y_pred, axes=1)
    Mf = tf.square(Mf)
    return tf.reduce_sum(Mf)

def einstein_Gravity_Min(K_A, y_pred, constant):
    Ka = tf.tensordot(K_A, y_pred, axes=1)
    Ka = Ka - constant
    Ka = tf.square(Ka)
    return tf.reduce_sum(Ka)

def Custom_Loss_Function(y_true, y_pred):   
    alpha = 10**2
    beta = 10**1

    # first term
    sum1 = crossing_Symmetry_Square(M, y_pred)
   
    #second term
    sum2 = einstein_Gravity_Min(K_A, y_pred, 1)
   
    #third term
    Kb = tf.tensordot(K_B, y_pred, axes=1)
    sum3 = Kb
    
    # Change sign on sum3 if we want to maximize (-) or minimize (+) wrt EG
    return sum1 + alpha*sum2 -beta*sum3 

# Testing

In [49]:
F2 = np.ones(4900)
F2 = F2/1000
F1 = np.random.rand(4900)
c  = 1
alpha = 10**10
beta = 10**20
Custom_Loss_Function(F1, F2).numpy()

7.368432560917732e+24